<a href="https://colab.research.google.com/github/nabatzis/ai-engg/blob/main/week3/deepseek_fine_tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install unsloth trl
!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [ ]:
import torch
from transformers import TrainingArguments
torch.cuda.is_available() and torch.cuda.is_bf16_supported()

from trl import SFTTrainer
from unsloth import FastLanguageModel
from huggingface_hub import login
import wandb
from getpass import getpass
import os
from datasets import load_dataset

# Prompt for tokens securely (input will be hidden)
print("Please enter your access tokens:")
hf_token = getpass('Enter your Hugging Face token: ')
wb_token = getpass('Enter your Weights & Biases token: ')

# Store them as environment variables for the session
os.environ['HUGGINGFACE_TOKEN'] = hf_token
os.environ['WANDB_API_KEY'] = wb_token

# Login to both platforms
login(hf_token)  # Hugging Face login
wandb.login(key=wb_token)  # W&B login

# Initialize W&B run
run = wandb.init(
    project='Fine-tune-DeepSeek-R1-Distill-Llama-8B on Medical COT Dataset',
    job_type="training",
    anonymous="allow"
)

Please enter your access tokens:
Enter your Hugging Face token: ··········
Enter your Weights & Biases token: ··········


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [ ]:
from unsloth import FastLanguageModel

max_seq_length = 2048
dtype = None
load_in_4bit = True


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/DeepSeek-R1-Distill-Llama-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = hf_token,
)

==((====))==  Unsloth 2025.1.8: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning.
Please answer the following medical question.

### Question:
{}

### Response:
<think>{}"""

In [ ]:
import time

question = "A 61-year-old woman with a long history of involuntary urine loss during activities like coughing or sneezing but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings, what would cystometry most likely reveal about her residual volume and detrusor contractions?"
FastLanguageModel.for_inference(model)

# Tokenize input
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

# Record start time
start_time = time.time()

# Generate
outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)

# Calculate time taken
end_time = time.time()
time_taken = end_time - start_time

# Calculate number of new tokens generated
input_length = inputs.input_ids.shape[1]
output_length = outputs.shape[1]
new_tokens = output_length - input_length

# Calculate tokens per second
tokens_per_sec = new_tokens / time_taken

print(f"Generation Speed: {tokens_per_sec:.2f} tokens/sec")
print(f"Total new tokens: {new_tokens}")
print(f"Time taken: {time_taken:.2f} seconds")

# Print the response
response = tokenizer.batch_decode(outputs)
print("\nResponse:")
print(response[0].split("### Response:")[1])

Generation Speed: 16.07 tokens/sec
Total new tokens: 704
Time taken: 43.80 seconds

Response:

<think>
Okay, so I'm trying to figure out what cystometry would show for this 61-year-old woman. She's been dealing with involuntary urine loss when she coughs or sneezes but doesn't leak at night. She had a gynecological exam and a Q-tip test. I need to connect these findings to what cystometry might reveal about her residual volume and detrusor contractions.

First, I should break down what's given. She has a history of urinary incontinence, specifically when doing activities that involve sudden coughing or sneezing. That makes me think of stress urinary incontinence because those are common triggers. But she doesn't leak at night, which might mean her bladder doesn't have significant leakage during sleep, possibly indicating that her bladder capacity isn't overly full at night.

She underwent a gynecological exam. I'm not exactly sure what the Q-tip test entails, but I think it's used to a

In [ ]:
train_prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning.
Please answer the following medical question.

### Question:
{}

### Response:
<think>
{}
</think>
{}"""

In [ ]:
EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN


def formatting_prompts_func(examples):
    inputs = examples["Question"]
    cots = examples["Complex_CoT"]
    outputs = examples["Response"]
    texts = []
    for input, cot, output in zip(inputs, cots, outputs):
        text = train_prompt_style.format(input, cot, output) + EOS_TOKEN
        texts.append(text)
    return {
        "text": texts,
    }

In [ ]:
from datasets import load_dataset
dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT","en", split = "train[0:500]",trust_remote_code=True)
dataset = dataset.map(formatting_prompts_func, batched = True,)
dataset["text"][0]

"Below is an instruction that describes a task, paired with an input that provides further context.\nWrite a response that appropriately completes the request.\nBefore answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.\n\n### Instruction:\nYou are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning.\nPlease answer the following medical question.\n\n### Question:\nA 61-year-old woman with a long history of involuntary urine loss during activities like coughing or sneezing but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings, what would cystometry most likely reveal about her residual volume and detrusor contractions?\n\n### Response:\n<think>\nOkay, let's think about this step by step. There's a 61-year-old woman here who's been dealing with involuntary urine leakages whenever she's doing something that ups her abdomi

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",  # True or "unsloth" for very long context
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=4,  # Increased from 2 to 4 for faster data processing
    args=TrainingArguments(
        per_device_train_batch_size=4,  # Increased from 2 to 4
        gradient_accumulation_steps=2,  # Reduced from 4 to 2
        warmup_steps=2,  # Reduced from 5
        max_steps=30,  # Reduced from 60
        learning_rate=5e-4,  # Increased from 2e-4 for faster convergence
        fp16=not torch.cuda.is_bf16_supported(),  # Option 1
        bf16=torch.cuda.is_bf16_supported(),      # Option 1
        logging_steps=5,  # Reduced from 10
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
    ),
)


Map (num_proc=4):   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 500 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 4 | Gradient Accumulation steps = 2
\        /    Total batch size = 8 | Total steps = 30
 "-____-"     Number of trainable parameters = 41,943,040
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
5,1.936400
10,1.494700
15,1.426200
20,1.380200
25,1.380400
30,1.368700


In [ ]:
question = "A 61-year-old woman with a long history of involuntary urine loss during activities like coughing or sneezing but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings, what would cystometry most likely reveal about her residual volume and detrusor contractions?"


FastLanguageModel.for_inference(model)  # Unsloth has 2x faster inference!
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])



<think>
Okay, let's see. This woman is 61, and she's been dealing with this issue of losing urine involuntarily for a while now. That sounds like a urinary incontinence problem. I know that incontinence can be due to several things, like weak muscles in the pelvic floor or nerve problems. 

When they mentioned the coughing or sneezing, that's a big clue. I remember that these kinds of activities can put a lot of pressure on the bladder, and it's often linked to stress incontinence. That's when the bladder can't hold the urine even when it's full, and it leaks when pressure is applied.

Now, about the exam and the Q-tip test. The Q-tip test is a quick way to check how well the bladder neck is supporting the bladder. If it's weak, it's more likely to leak under pressure. So if the exam shows that the bladder neck isn't holding up well, that's a sign that stress incontinence is a possibility.

Cystometry, or bladder diary, can show how much residual volume is left in the bladder after ur

In [ ]:
new_model_local = "DeepSeek-R1-Medical-COT-RV"
moal)del.save_pretrained(new_model_local)
tokenizer.save_pretrained(new_model_loc

model.save_pretrained_merged(new_model_local, tokenizer, save_method = "merged_16bit",)

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 6.0G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 2.4 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


 34%|███▍      | 11/32 [00:00<00:01, 12.35it/s]
We will save to Disk and not RAM now.
100%|██████████| 32/32 [04:07<00:00,  7.73s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving DeepSeek-R1-Medical-COT-RV/pytorch_model-00001-of-00004.bin...
Unsloth: Saving DeepSeek-R1-Medical-COT-RV/pytorch_model-00002-of-00004.bin...
Unsloth: Saving DeepSeek-R1-Medical-COT-RV/pytorch_model-00003-of-00004.bin...
Unsloth: Saving DeepSeek-R1-Medical-COT-RV/pytorch_model-00004-of-00004.bin...
Done.


In [ ]:
new_model_online = "goaiguru/DeepSeek-R1-Medical-COT-RV"
model.push_to_hub(new_model_online)
tokenizer.push_to_hub(new_model_online)

model.push_to_hub_merged(new_model_online, tokenizer, save_method = "merged_16bit")

README.md:   0%|          | 0.00/633 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Saved model to https://huggingface.co/goaiguru/DeepSeek-R1-Medical-COT-RV


No files have been modified since last commit. Skipping to prevent empty commit.
Unsloth: You are pushing to hub, but you passed your HF username = goaiguru.
We shall truncate goaiguru/DeepSeek-R1-Medical-COT-RV to DeepSeek-R1-Medical-COT-RV
Unsloth: Will remove a cached repo with size 1.6K


...

pytorch_model-00003-of-00004.bin:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

pytorch_model-00002-of-00004.bin:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

pytorch_model-00001-of-00004.bin:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Done.
Saved merged model to https://huggingface.co/goaiguru/DeepSeek-R1-Medical-COT-RV
